# Playground for ASFPy module development

In [53]:
import os
from operator import itemgetter

import asfpy

In [61]:
applicants = [
    {
        "id": "APP001",
        "urm": False,
        "lim": False,
        "du": True,
        "categories": {
            "Clinical"
        },
    },
    {
        "id": "APP002",
        "urm": True,
        "lim": False,
        "du": True,
        "categories": {
            "Cognitive"
        },
    },
    {
        "id": "APP003",
        "urm": True,
        "lim": False,
        "du": True,
        "categories": {
            "Affective",
            "Cognitive"
        }
    },
    {
        "id": "APP004",
        "urm": True,
        "lim": True,
        "du": False,
        "categories": {
            "Clinical"
        }
    },
    {
        "id": "APP005",
        "urm": False,
        "lim": True,
        "du": False,
        "categories": {
            "Developmental"
        }
    }
]

In [11]:
categories = [
    "Clinical",
    "Developmental",
    "Affective",
    "Social",
    "Cognitive"
]

In [26]:
def rank(applicant):
    """
    Rank an applicant by attribute combinations.
    """
    is_urm = applicant["urm"]
    is_lim = applicant["lim"]
    is_du = applicant["du"]
    
    if (is_urm and is_lim and is_du):
        rank = 0
    elif (is_urm and is_lim):
        rank = 1
    elif (is_urm or is_lim) and is_du:
        rank = 2
    elif (is_urm or is_lim):
        rank = 3
    elif is_du:
        rank = 4
    else:
        rank = 5
        
    return rank

In [62]:
def prioritize(applicants):
    """
    Prioritize applicants by rank of attributes.
    """
    for a in applicants:
        a["rank"] = rank(a)
    applicants.sort(key = itemgetter("rank"))
    return applicants

In [63]:
prioritize(applicants)

[{'id': 'APP004',
  'urm': True,
  'lim': True,
  'du': False,
  'categories': {'Clinical'},
  'rank': 1},
 {'id': 'APP002',
  'urm': True,
  'lim': False,
  'du': True,
  'categories': {'Cognitive'},
  'rank': 2},
 {'id': 'APP003',
  'urm': True,
  'lim': False,
  'du': True,
  'categories': {'Affective', 'Cognitive'},
  'rank': 2},
 {'id': 'APP005',
  'urm': False,
  'lim': True,
  'du': False,
  'categories': {'Developmental'},
  'rank': 3},
 {'id': 'APP001',
  'urm': False,
  'lim': False,
  'du': True,
  'categories': {'Clinical'},
  'rank': 4}]